In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,2):
    print 'scrapping page : ', i
    url = 'http://www.hawkerfood.com/page/' + str(i) + '/'  

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('div', {'class' : 'blog-item-wrap'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('time', {'class' : 'entry-date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "entry-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  http://www.hawkerfood.com/chin-mee-chin-confectionery-katong-east-coast-road/
cleaned detail snippet :  i am pretty sure many of you have visited chin mee chin confectionery locat
href :  http://www.hawkerfood.com/sheng-seng-fried-prawn-noodle-pek-kio-market-food-centre/
cleaned detail snippet :  food prices are always on the rise so it is hard to find something that is 
href :  http://www.hawkerfood.com/engs-noodles-house-wanton-mee-tanjong-katong/
cleaned detail snippet :  a few months back i wandered to tanjong katong to jalanjalan and to look fo
href :  http://www.hawkerfood.com/ponggol-nasi-lemak-upper-serangoon-road/
cleaned detail snippet :  ponggol or punggol is literally the same word with different spelling but m
href :  http://www.hawkerfood.com/armenian-street-char-kway-teow-anchorvale-sengkang/
cleaned detail snippet :  armenian street char kway teow ya mi ni ya jie chao guo tiao has a history 
href :  http://www.hawkerfood.com/indonesian-ayam-b

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.13643693924  seconds
10
10


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 7.94810390472  seconds
10
10


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
5,\n\n \n\n\nIndonesian Ayam Bakar in Changi Vil...,There are not many stalls in Singapore dedicat...,"May 14, 2017",indonesian ayam bakar changi village hawker la...,stall dedicated selling ayam bakar indonesian ...
6,\n\n \n\n\n328 Katong Laksa (328 加东叻沙) along E...,"When you mention Laksa, 328 Katong Laksa (328 ...","April 23, 2017",katong laksa jia dong le sha east coast road a...,mention laksa katong laksa jia dong le sha eas...
7,\n\n \n\n\nTsuru-Koshi Udon in Takashimaya Sho...,Takashimaya Shopping Centre Basement 2 is a ni...,"March 25, 2017",tsurukoshi udon takashimaya shopping march lao...,takashimaya shopping basement nice genre espec...
8,\n\n \n\n\nHong Heng Beef Noodle Soup and Laks...,It is a known fact that some hawker stalls in ...,"February 21, 2017",hong heng beef noodle soup laksa kebun baru fe...,known fact hawker stall operate end lunchtime ...
9,\n\n \n\n\nInternational Muslim Food Stall Nas...,Lunchtime Scenario: You walk into a hawker cen...,"January 18, 2017",international muslim stall nasi lemak changi v...,lunchtime scenario walk hawker craving nasi le...


In [8]:
len(df)

10

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

10

In [11]:
x = df.iloc[3,1]
print x

Ponggol or Punggol is literally the same word with different spelling but most of us are more familiar with the latter. Thus, when I first saw the name Ponggol Nasi Lemak, I was kind of intrigued. What was even more intriguing to me initially was the long queue for its nasi lemak during peak dinner hours.   To avoid the crowd, I was there right at their opening time of 5.30pm and a queue has already formed. Most were ordering takeaways with a handful of them dining in. Before I joined the queue, I went to browse the dishes and saw a wide variety of them like ikan bilis, ikan kuning, fried chicken wing, luncheon meat, fish fillet, otah, an assortment of vegetables, etc.   The queue was moving pretty fast and probably because none in front of me made a bulk order which was kind of a relief to me. I ordered curry vegetables, egg, fried chicken wing and three pieces of ikan kuning (Yellow-banded Scad). As always, the senile uncle in me forgot to take note of the price but I remember it was

In [12]:
# pickle dataframe
df.to_pickle('hawkerfood.pickle')

In [13]:
wd.quit()